# Socket编程发展

## select模型

```c
int select(int nfds, fd_set *readfds, fd_set *writefds, fd_set *exceptfds, struct timeval *timeout);

void FD_CLR(int fd, fd_set *set);
int  FD_ISSET(int fd, fd_set *set);
void FD_SET(int fd, fd_set *set);
void FD_ZERO(fd_set *set);
```

```c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <sys/types.h>
#include <unistd.h>

int
main(void)
{
   fd_set rfds;
   struct timeval tv;
   int retval;

   /* Watch stdin (fd 0) to see when it has input. */

   FD_ZERO(&rfds);
   FD_SET(0, &rfds);

   /* Wait up to five seconds. */

   tv.tv_sec = 5;
   tv.tv_usec = 0;

   retval = select(1, &rfds, NULL, NULL, &tv);
   /* Don't rely on the value of tv now! */

   if (retval == -1)
       perror("select()");
   else if (retval)
       printf("Data is available now.\n");
       /* FD_ISSET(0, &rfds) will be true. */
   else
       printf("No data within five seconds.\n");

   exit(EXIT_SUCCESS);
}
```

select函数监视的文件描述分为3类，分别为`readfds`、`writefds`和`exceptfds`，调用select函数后会阻塞，直到有文件描述就绪（有数据可读、可写、异常）或者超时，当函数返回后，通过遍历fd_sets来找到就绪的描述符。

select缺点在于单个进程监视的文件描述符的数量存在最大限制，Linux上一般为1024。

## epoll模型

```c
int epoll_create(int size);
int epoll_ctl(int epfd, int op, int fd, struct epoll_event *event);
int epoll_wait(int epfd, struct epoll_event *events, int maxevents, int timeout);
```

```c
typedef union epoll_data {
    void *ptr;
    int fd;
    __uint32_t u32;
    __uint64_t u64;
} epoll_data_t;
struct epoll_event {
    __uint32_t events;
    epoll_data_t data;
};
```

* `epoll_create()`创建epoll对象，`size`指定监听文件描述符的最大数量；
* `epoll_ctl()`是fd的添加、删除和更新操作；
* `epoll_wait()`是等待epfd上的IO事件，最大返回`maxevents`个事件。

`epoll`主要的优点：

* 监听文件描述不受限制，它支持`fd`的上限是最大可打开文件的数目（可以通过 `cat /proc/sys/fs/file-max`查看）
* IO效率不会随着监视fd数量的增长而下降，`epoll`不同于`select`轮询的方式，而是通过每个fd定义的回调函数来实现，只有就绪的fd才会执行回调函数；
* 支持水平触发和边缘触发两种方式：
    * 水平触发，只有fd对应缓冲区有数据，就会反复触发，编程简单；
    * 边缘触发，fd的缓冲区如果有读取的数据，下次就绪前，不会被再次触发，效率更高，编程复杂。
* mmap避免了内存拷贝，加速内核和用户空间的信息传递。